In [1]:
#General modules
import numpy as np
import numpy.linalg as la
import pandas as pd
import matplotlib.pyplot as plt

# Text processing modules
import string
import re
import nltk
nltk.download('punkt')

# ML modules
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import BernoulliNB
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix

# Preprocessing functions
def text_preprocessing(text, stop_words):
    """
    Performs tokenization and simple preprocessing.
    Args : 
        @param text (String): Text that needs preprocessing
        @return: text (String) : Preprocessed text
    """
    bad_symbols = re.compile('[/(){}\[\]\|@,;+_#]')
    fine_symbols = re.compile('[^0-9абвгдѓежзѕијклљмнњопрстќуфхцчџш ]')
    stopwords_set = stop_words
    lat_to_cyr = {'kj' : 'ќ', 'gj' : 'ѓ', 'zh' : 'ж', 'ch' : 'ч', 'sh' : 'ш', 'dj' : 'ѓ',
              'a' : 'а', 'b' : 'б', 'c' : 'ц', 'd' : 'д', 'e' : 'е', 'f' : 'ф', 'g' : 'г',
              'h' : 'х', 'i' : 'и', 'j' : 'ј', 'k' : 'к', 'l' : 'л', 'm' : 'м', 'n' : 'н', 
              'o' : 'о', 'p' : 'п', 'q' : 'љ', 'r' : 'р', 's' : 'с', 't' : 'т', 'u' : 'у',
              'v' : 'в', 'w' : 'њ', 'x' : 'џ', 'y' : 'ѕ',  'z' : 'з'
             }
    
    text = text.lower()
    
    for item, value in lat_to_cyr.items():
        text = re.sub(item, value, text)
    
    text = bad_symbols.sub(' ', text)

    text = fine_symbols.sub('', text)
    
    text = ' '.join([x for x in text.split() if x and x not in stopwords_set])

    return text.strip()



# Load the dataset
def load_dataset(source):
    # Loading the data from the source
    data = pd.read_csv(source)

    # Spliting into questions, answers and categories
    questions = data.iloc[:, 0]
    answers = data.iloc[:, 1]
    categories = data.iloc[:, 2]
    category_codes = data.iloc[:, 3]
    
    return questions, answers, categories, category_codes


# Load the stop words
def load_stop_words(source):
    file = open(source, 'r')
    sw=file.read().split()
    file.close()
    return sw

[nltk_data] Downloading package punkt to /home/filip/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
# Loading the database of questions and building the corpus
qus, ans, cat, cat_codes = load_dataset('dataset_brainster.csv')
sw = load_stop_words('stop_words.txt')

corpus = [text_preprocessing(q, sw) for q in qus]

In [3]:
corpus[50:60]

['сео',
 'гроњтх хацкинг',
 'сторѕтеллинг',
 'цопѕњритинг',
 'леад генератион',
 'време трае академијата графички дизајн',
 'трае академијата графички дизајн',
 'времетрање академијата графички дизајн',
 'одвива наставата академијата графички дизајн',
 'наставата академијата графички дизајн одвива живо онлајн']

In [4]:
# Builting the Count-Vectorized features
count_vectorizer = CountVectorizer(strip_accents='unicode') #probaj bez max_features
count_vectorizer.fit(corpus)
corpus_vec = count_vectorizer.transform(corpus)

corpus_vec_df = pd.DataFrame(data = corpus_vec.todense(), columns = count_vectorizer.get_feature_names())

In [5]:
corpus_vec_df.head()

,15,1630,24,автоматско,адс,академии,академиите,академија,академијата,акредитиран,...,чини,школувањето,јава,јазик,јазици,јак,јљуерѕ,ља,њебдривер,њебсите
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Building the TF-IDF Vectorized Features
tfidf_vectorizer = TfidfVectorizer(strip_accents='unicode')
tfidf_vectorizer.fit(corpus)
corpus_tfidif = tfidf_vectorizer.transform(corpus)

corpus_tfidif_df = pd.DataFrame(data = corpus_tfidif.todense(), columns = tfidf_vectorizer.get_feature_names())

In [7]:
corpus_tfidif_df.shape

(523, 499)

## Testing CountVectorizer

In [88]:
# Defining Cosine Similarity Functiom
def cos_sim(input, corpus): 
    cosine_function = lambda a, b : np.inner(a, b)/(np.linalg.norm(a)*np.linalg.norm(b))
    distances = []
    for question in corpus:
        distances.append(cosine_function(input.todense(), question.todense()))
    max_sim = max(distances)
    max_arg = np.argmax(distances)
    return max_arg, max_sim, distances

In [89]:
# Defining Pearson Similarity Function
def pearson_sim(input, corpus): 
    distances = []
    for question in corpus:
        distances.append(np.corrcoef(input.todense(), question.todense())[0, 1])
    max_sim = max(distances)
    max_arg = np.argmax(distances)
    return max_arg, max_sim, distances

In [87]:
tq = 'дали нудите курсеви за програмирање?'
tq = text_preprocessing(tq, sw)
tq = count_vectorizer.transform([tq])

In [90]:
# Getting the closest cosine-score and responding
max_arg, max_sim, distances = cos_sim(tq, corpus_vec)

print(max_arg)

print(max_sim)

print('Најблиску до прашањето:', qus[max_arg])
print('Одговор: ', ans[max_arg])

89
[[0.57735027]]
Најблиску до прашањето: Дали нудите курсеви за графички дизајн?
Одговор:  Во моментов курсевите не се одржуваат.


In [91]:
# Getting the closest pearson-score and responding
max_arg, max_sim, distances = pearson_sim(tq, corpus_vec)

print(max_arg)

print(max_sim)

print('Најблиску до прашањето:', qus[max_arg])
print('Одговор: ', ans[max_arg])

89
0.5744375935906015
Најблиску до прашањето: Дали нудите курсеви за графички дизајн?
Одговор:  Во моментов курсевите не се одржуваат.


## Testing TF-IDF Vectorizer

In [74]:
tq = 'колку кошта академијата за дата сциенце ако се плати на рати?'
tq = text_preprocessing(tq, sw)
tq = count_vectorizer.transform([tq])

In [85]:
# Getting the closest cosine-score and responding
max_arg, max_sim, distances = cos_sim(tq, corpus_tfidif)

print(max_arg)

print(max_sim)

print('Најблиску до прашањето:', qus[max_arg])
print('Одговор: ', ans[max_arg])

365
[[0.63245553]]
Најблиску до прашањето: Што е Data Science?
Одговор:  Data Science е интердисциплинарно поле во кое се применуваат научни методи, процеси, алгоритми и системи за извлекување на корисно знаење и информации од структуирани и неструктуирани податоци.  Data Science  подразбира примена на знаење од областса на Machine learning, Python, Big Data, Business Inteligence, SQL, математика и статистика.


In [86]:
# Getting the closest pearson-score and responding
max_arg, max_sim, distances = pearson_sim(tq, corpus_tfidif)

print(max_arg)

print(max_sim)

print('Најблиску до прашањето:', qus[max_arg])
print('Одговор: ', ans[max_arg])

365
0.6305438232815118
Најблиску до прашањето: Што е Data Science?
Одговор:  Data Science е интердисциплинарно поле во кое се применуваат научни методи, процеси, алгоритми и системи за извлекување на корисно знаење и информации од структуирани и неструктуирани податоци.  Data Science  подразбира примена на знаење од областса на Machine learning, Python, Big Data, Business Inteligence, SQL, математика и статистика.
